In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np
from datetime import datetime, date, time
mdata = sio.loadmat("DREAMER.mat")
mdata = mdata['DREAMER']
mdtype = mdata.dtype 


In [2]:
def read_data(dataset):
    matrix_data = np.zeros((18, 128*60*14 + 3))
    count_row=0
    for elem in dataset:
        i=0
        row = []
        elem_T = elem[0].T
        for i in range(0,14):
            counter = 1
            for num in elem_T[i]:
                i=i+1
                if counter <= 128*60:
                    row.append(num)
                    counter = counter + 1
        matrix_data[count_row][:-3] = row
        count_row = count_row + 1
    return matrix_data

In [3]:

ndata = {n: mdata[n][0, 0] for n in mdtype.names}
ndata = ndata['Data']
ndata= ndata.reshape((23,))
matrix_data = np.zeros((23*18, 128*60*14 + 3))
i=0
for data in ndata:
    row = []
    EEG = data['EEG'][0][0][0][0]
    baseline = EEG['baseline']
    stimuli = EEG['stimuli'] 
    #Dentro matrix data una riga è un singolo esperimento con tutti e 14 
    #gli elettrodi concatenati (baseline già sottratta). Per ogni paziente abbiamo 18 righer
    matrix_data[i:i+18] = read_data(stimuli) - read_data(baseline)
    i=i+18
    

In [4]:
valence_all=[]
arousal_all = []
dominance_all = []
for data in ndata:
    valence = data['ScoreValence'][0][0]
    arousal = data['ScoreArousal'][0][0]
    dominance = data['ScoreDominance'][0][0]
    for elem in valence:
        valence_all.append(elem[0])
    for elem in arousal:
        arousal_all.append(elem[0])
    for elem in dominance:
        dominance_all.append(elem[0])
    


In [5]:
i=0
while i<len(valence_all):
    matrix_data[i][128*60*14] = valence_all[i]
    matrix_data[i][128*60*14 + 1] = arousal_all[i]
    matrix_data[i][128*60*14 + 2] = dominance_all[i]
    i = i + 1

In [6]:
print(matrix_data[50])

[-63.07692308 -39.48717949 -68.71794872 ...   4.           2.
   3.        ]


In [9]:
df = pd.DataFrame(matrix_data).T
#df.to_excel(excel_writer = "DREAMR.xlsx")
df.to_csv("DREAMR.csv")